In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
num_features=784

x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

x_train, x_test = x_train / 255., x_test / 255.

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
num_classes = 10

learning_rate = 0.01

training_steps = 1000

batch_size = 256

display_step = 50

In [4]:
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))

train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [5]:
W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")

b = tf.Variable(tf.zeros([num_classes]), name="bias")

In [6]:
def logistic_regression(x):

    return tf.nn.softmax(tf.matmul(x, W) + b)

def cross_entropy(y_pred, y_true):

    y_true = tf.one_hot(y_true, depth=num_classes)


    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)

    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

In [7]:
def accuracy(y_pred, y_true):

    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))

    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

optimizer = tf.optimizers.SGD(learning_rate)

In [8]:
def run_optimization(x, y):

    with tf.GradientTape() as g:

        pred = logistic_regression(x)

        loss = cross_entropy(pred, y)

    gradients = g.gradient(loss, [W, b])

    optimizer.apply_gradients(zip(gradients, [W, b]))

In [9]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):

    run_optimization(batch_x, batch_y)

    if step % display_step == 0:

        pred = logistic_regression(batch_x)

        loss = cross_entropy(pred, batch_y)

        acc = accuracy(pred, batch_y)

        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 114.008575, accuracy: 0.851562
step: 100, loss: 664.286499, accuracy: 0.812500
step: 150, loss: 145.496567, accuracy: 0.851562
step: 200, loss: 97.200005, accuracy: 0.921875
step: 250, loss: 81.006561, accuracy: 0.921875
step: 300, loss: 68.176468, accuracy: 0.929688
step: 350, loss: 151.487976, accuracy: 0.847656
step: 400, loss: 149.583710, accuracy: 0.851562
step: 450, loss: 40.089024, accuracy: 0.957031
step: 500, loss: 113.306625, accuracy: 0.898438
step: 550, loss: 61.312424, accuracy: 0.921875
step: 600, loss: 145.479126, accuracy: 0.843750
step: 650, loss: 215.785583, accuracy: 0.792969
step: 700, loss: 168.112823, accuracy: 0.867188
step: 750, loss: 70.123138, accuracy: 0.929688
step: 800, loss: 29.680958, accuracy: 0.960938
step: 850, loss: 74.179489, accuracy: 0.902344
step: 900, loss: 66.798889, accuracy: 0.917969
step: 950, loss: 44.981163, accuracy: 0.953125
step: 1000, loss: 55.124702, accuracy: 0.933594


In [10]:
pred = logistic_regression(x_test)

print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.922800
